# crypto token correllation Heatmap

In [2]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd

load_dotenv()  # Load environment variables from .env

api_key = os.getenv("GECKO_API")  # Get API key from .env
coin_id = "bitcoin"
vs_currency = "usd"
days = "30"

url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
params = {
    "vs_currency": vs_currency,
    "days": days
}
headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": api_key
}

response = requests.get(url, headers=headers, params=params)
print(response.text)


{"status":{"error_code":10002,"error_message":"API Key Missing"}}


In [3]:
print("API Key:", api_key)


API Key: None


In [4]:
names = os.getenv("name")

print(names)


None
